In [14]:
import os
import pandas as pd
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache
import brain_observatory_utilities.datasets.optical_physiology.data_formatting as ophys_formatting
import brain_observatory_utilities.utilities.general_utilities as utilities
import pickle

In [2]:
datafolder = "D:\\neuromatch2024\\datasets\\project\\Allen\\temp\\"      ### Please rename this appropriately
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=datafolder)

ophys_session_id = 854060305
ophys_experiment_ids = [854759890, 854759894, 854759896, 854759898]

experiments = {}
# ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
for ophys_experiment_id in ophys_experiment_ids:
    experiments[ophys_experiment_id] = cache.get_behavior_ophys_experiment(
        ophys_experiment_id
    )

c:\Users\bosem\miniconda3\envs\allenSDK\lib\site-packages\allensdk\brain_observatory\behavior\behavior_project_cache\behavior_project_cache.py:135: UpdatedStimulusPresentationTableWarning: 
	As of AllenSDK version 2.16.0, the latest Visual Behavior Ophys data has been significantly updated from previous releases. Specifically the user will need to update all processing of the stimulus_presentations tables. These tables now include multiple stimulus types delineated by the columns `stimulus_block` and `stimulus_block_name`.

The data that was available in previous releases are stored in the block name containing 'change_detection' and can be accessed in the pandas table by using: 
	`stimulus_presentations[stimulus_presentations.stimulus_block_name.str.contains('change_detection')]`
  warnings.warn(
c:\Users\bosem\miniconda3\envs\allenSDK\lib\site-packages\hdmf\utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  retur

## First thing to extract : neural data

In [4]:
neural_data = []
for ophys_experiment_id in ophys_experiment_ids:
    this_experiment = experiments[ophys_experiment_id]
    this_experiment_neural_data = ophys_formatting.build_tidy_cell_df(this_experiment)

    # add some columns with metadata for the experiment
    metadata_keys = [
        "ophys_experiment_id",
        "ophys_session_id",
        "targeted_structure",
        "imaging_depth",
        "equipment_name",
        "cre_line",
        "mouse_id",
        "sex",
    ]
    for metadata_key in metadata_keys:
        this_experiment_neural_data[metadata_key] = this_experiment.metadata[
            metadata_key
        ]

    # append the data for this experiment to a list
    neural_data.append(this_experiment_neural_data)

# concatate the list of dataframes into a single dataframe
neural_data = pd.concat(neural_data)

## Next up : stimulus table

In [5]:
stimulus_table = experiments[ophys_experiment_id].stimulus_presentations.reset_index()

## Next : licks

In [6]:
licks = experiments[ophys_experiment_id].licks

## Next : rewards

In [7]:
rewards = experiments[ophys_experiment_id].rewards

## Next : eye-tracking

In [11]:
eyetracking = experiments[ophys_experiment_id].eye_tracking
eyetracking.head()

,timestamps,cr_area,eye_area,pupil_area,likely_blink,pupil_area_raw,cr_area_raw,eye_area_raw,cr_center_x,cr_center_y,...,eye_center_x,eye_center_y,eye_width,eye_height,eye_phi,pupil_center_x,pupil_center_y,pupil_width,pupil_height,pupil_phi
frame,,,,,,,,,,,,,,,,,,,,,
0,0.18027,144.263415,56298.679582,19017.685847,False,19017.685847,144.263415,56298.679582,295.938814,192.182795,...,312.355350,177.187174,150.051302,119.428663,0.015626,299.778479,180.005770,76.802907,77.804353,-0.126717
1,0.22745,150.307573,56344.170996,18952.196738,False,18952.196738,150.307573,56344.170996,295.832810,192.124258,...,312.128961,177.364960,149.885210,119.657614,0.015203,299.739360,180.439130,77.113321,77.670275,0.546624
2,0.23823,147.560588,56353.694978,19073.536724,False,19073.536724,147.560588,56353.694978,295.841372,192.432613,...,312.651973,177.346389,149.785008,119.757902,0.019766,300.175846,180.451799,77.918517,75.153499,-0.391708
3,0.25756,143.196925,56643.476983,19883.989112,False,19883.989112,143.196925,56643.476983,296.302748,192.472649,...,312.577134,177.406775,149.561546,120.553572,0.007777,300.404457,179.203032,79.556711,76.418081,-0.748424
4,0.29091,133.938976,56614.648388,20897.926610,False,20897.926610,133.938976,56614.648388,296.381275,192.800057,...,313.687843,177.948115,149.483529,120.555103,0.011993,300.621451,176.434152,76.154457,81.559896,0.455449


## Next : running speed

In [13]:
running = experiments[ophys_experiment_id].running_speed
running

,timestamps,speed
0,10.53881,0.001962
1,10.55547,0.139393
2,10.57216,0.258182
3,10.58882,0.341831
4,10.60551,0.382968
...,...,...
270195,4517.46887,-0.310378
270196,4517.48557,-0.742260
270197,4517.50225,-1.298409
270198,4517.51889,-1.953897


## Now create one single dict to hold them all

In [15]:
masterdict = {
    "neural_data" : neural_data,
    "stimulus_table" : stimulus_table,
    "licks" : licks,
    "rewards" : rewards,
    "eyetracking" : eyetracking,
    "running" : running
}

with open(f'{datafolder}masterfile_{ophys_session_id}.pickle', 'wb') as handle:
    pickle.dump(masterdict, handle)

## You can read back the file as follows

In [16]:
with open(f'{datafolder}masterfile_{ophys_session_id}.pickle', 'rb') as handle:
    masterdict = pickle.load(handle)

In [17]:
masterdict

{'neural_data':        timestamps       dff  events  filtered_events  cell_roi_id  \
 0        10.52216  0.400583     0.0     0.000000e+00   1080852071   
 1        10.61538  0.126125     0.0     0.000000e+00   1080852071   
 2        10.70860 -0.083087     0.0     0.000000e+00   1080852071   
 3        10.80182  0.158960     0.0     0.000000e+00   1080852071   
 4        10.89504  0.301507     0.0     0.000000e+00   1080852071   
 ...           ...       ...     ...              ...          ...   
 48326  4517.18670  0.330930     0.0     1.557620e-63   1080852363   
 48327  4517.27996  0.142227     0.0     2.591872e-74   1080852363   
 48328  4517.37322  0.195335     0.0     5.921853e-86   1080852363   
 48329  4517.46647  0.182008     0.0     1.857778e-98   1080852363   
 48330  4517.55973  0.103122     0.0    8.002437e-112   1080852363   
 
        cell_specimen_id  ophys_experiment_id  ophys_session_id  \
 0            1086550481            854759890         854060305   
 1       

In [18]:
masterdict.keys()

dict_keys(['neural_data', 'stimulus_table', 'licks', 'rewards', 'eyetracking', 'running'])